# 1D Ising model

$$
H = -J\sum_{j=1}^{L} s_j s_{j+1} \,,
$$

on a chain of $L$ sites with PBC.

## Partition function and other observables

Using e.g. R. Swendsen, Chap 30, the partition function is

$$
Z = (2\cosh{\beta J})^L + (2\sinh{\beta J})^L
$$

Therefore internal energy is 
$$
E = -\frac{\partial}{\partial \beta}  \ln Z = -L x \frac{1 + x^{L-2}}{1 + x^L} \,,
$$

where $x = \tanh{\beta J}$.

# A small chain of $L=4$ sites

Can enumerate the spin states ($2^4 = 16$ states in total). 

## First, set up the lattice.

In [21]:
import itertools
from math import exp
import numpy as np

from mc_lib.lattices import tabulate_neighbors

In [22]:
L = 4
Nsite = L
neighbors = tabulate_neighbors((L, 1, 1), kind='sc')
neighbors

array([[2, 1, 3],
       [2, 0, 2],
       [2, 1, 3],
       [2, 0, 2]])

Internally, the sites are indexed with a flat index (in higher dimensions, too). The format of the `neighbors` array is that `num = neighbors[site, 0]` is the number of neighbors of the site `site`, and `neighbors[site, 1:num]` are the sites which are neighbors of `site`. For instance, the output above means that the site number zero has two neighbors, sites `1` and `3`, site `1` is connected to sites `0` and `2` and so on.

## Enumerate the spin states on the lattice

In [23]:
site_basis = (-1, 1)
states = [_ for _ in itertools.product(*[site_basis]*Nsite)]
states

[(-1, -1, -1, -1),
 (-1, -1, -1, 1),
 (-1, -1, 1, -1),
 (-1, -1, 1, 1),
 (-1, 1, -1, -1),
 (-1, 1, -1, 1),
 (-1, 1, 1, -1),
 (-1, 1, 1, 1),
 (1, -1, -1, -1),
 (1, -1, -1, 1),
 (1, -1, 1, -1),
 (1, -1, 1, 1),
 (1, 1, -1, -1),
 (1, 1, -1, 1),
 (1, 1, 1, -1),
 (1, 1, 1, 1)]

## Define the Hamiltonian

Given a spin state (an element of the `states` list above), the Ising energy is the sum over all bonds of the lattice (i.e. a pair of neighbor sites).

In [24]:
def energy(spins, neighbors, J=1.0):
    """Ising model energy of a spin state.
    """
    ene = 0
    for site in range(Nsite):
        num_neighb = neighbors[site, 0]
        for site1 in neighbors[site, 1:num_neighb+1]:
            ene += -J * spins[site] * spins[site1]
    
    # each bond is counted twice, hence divide by two
    return ene / 2.0

In [25]:
for state in states:
    print(state, energy(state, neighbors))

(-1, -1, -1, -1) -4.0
(-1, -1, -1, 1) 0.0
(-1, -1, 1, -1) 0.0
(-1, -1, 1, 1) 0.0
(-1, 1, -1, -1) 0.0
(-1, 1, -1, 1) 4.0
(-1, 1, 1, -1) 0.0
(-1, 1, 1, 1) 0.0
(1, -1, -1, -1) 0.0
(1, -1, -1, 1) 0.0
(1, -1, 1, -1) 4.0
(1, -1, 1, 1) 0.0
(1, 1, -1, -1) 0.0
(1, 1, -1, 1) 0.0
(1, 1, 1, -1) 0.0
(1, 1, 1, 1) -4.0


## Thermodynamic averages

We sum over the spin states with Gibbs weights, $\propto \exp{-E / T}$. 

(the summation below should probably use `math.fsum` to avoid loss of precision due to exponentially small and large terms).

In [26]:
# Average energy

def average_energy(states, neighbors, T):
    num, den = 0., 0.
    for state in states:
        ene = energy(state, neighbors, 1.0)
        weight = exp(-ene / T)
        num += ene * weight
        den += weight
    return num / den

## Compute the average energy

And compare to the exact answer.

In [35]:
Ts = np.linspace(0.2, 2.0, 19)
E = np.asarray([average_energy(states, neighbors, T) for T in Ts])
for t, e in zip(Ts, E):
    print(t, e, '    beta = ', 1./t)

0.2 -3.9999999505323136     beta =  5.0
0.30000000000000004 -3.9999611300337223     beta =  3.333333333333333
0.4 -3.9989106819285     beta =  2.5
0.5 -3.99196417187338     beta =  2.0
0.6000000000000001 -3.96967543351698     beta =  1.6666666666666665
0.7 -3.9222876735924705     beta =  1.4285714285714286
0.8 -3.8442305425947123     beta =  1.25
0.9000000000000001 -3.7357085101084806     beta =  1.111111111111111
1.0 -3.6016507256995927     beta =  1.0
1.1 -3.449503692064762     beta =  0.9090909090909091
1.2 -3.2871347967237092     beta =  0.8333333333333334
1.3 -3.1214540260541392     beta =  0.7692307692307692
1.4000000000000001 -2.957803300728428     beta =  0.7142857142857142
1.5 -2.7998930594102216     beta =  0.6666666666666666
1.6 -2.6500274633267966     beta =  0.625
1.7 -2.5094273927475785     beta =  0.5882352941176471
1.8 -2.37854404850287     beta =  0.5555555555555556
1.9000000000000001 -2.257318031086368     beta =  0.5263157894736842
2.0 -2.145374415963298     beta =  

In [33]:
Ts

array([0.1 , 0.29, 0.48, 0.67, 0.86, 1.05, 1.24, 1.43, 1.62, 1.81, 2.  ])

### Implement the exact formula (written out at the top of the notebook)

In [36]:
th = np.tanh(1./Ts)

ee = -L * th * (1 + th**(L-2)) / (1. + th**L)
ee

array([-3.99999995, -3.99996113, -3.99891068, -3.99196417, -3.96967543,
       -3.92228767, -3.84423054, -3.73570851, -3.60165073, -3.44950369,
       -3.2871348 , -3.12145403, -2.9578033 , -2.79989306, -2.65002746,
       -2.50942739, -2.37854405, -2.25731803, -2.14537442])

In [37]:
# compare 
E - ee

array([-4.4408921e-16, -8.8817842e-16,  0.0000000e+00, -8.8817842e-16,
        0.0000000e+00, -4.4408921e-16,  8.8817842e-16,  4.4408921e-16,
       -4.4408921e-16,  0.0000000e+00,  4.4408921e-16,  4.4408921e-16,
        0.0000000e+00,  4.4408921e-16, -4.4408921e-16,  4.4408921e-16,
       -4.4408921e-16,  0.0000000e+00,  0.0000000e+00])

In [25]:
th = np.tanh(0.25)

ee = -4 * th * (1 + th**(L-2)) / (1. + th**L)
ee

-1.0347174422904286

# MC simulation

From the terminal, run

```
> python setup.py build_ext -i
> python -c'python -c'from cy_ising import simulate; simulate(L=4, beta=1.25, num_sweeps=100000)'
```